### Import libraries

In [ ]:
import os
os.chdir("")
print("Current working dir : %s" % os.getcwd())

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk

### Train models of each user for each day using multiple algorithms

In [ ]:
# define variables
days = [13,14,15,16,17,22]
users = [94,95,97,98,99,100,101,102,103,104,105]
accuracy = []
lst_dict = []

# train by each user
for user in users:
    
    print("-------------------", user ,"-----------------------------")   
    
    # prepare data
    data = pd.read_csv("data/train_data_"+str(user)+".csv");
    data.time = pd.to_datetime(data.time)
    data['day'] = data.time.dt.day
    data = data.reset_index()
    data = data.dropna()
     
    # train by each day
    for day in days:
        
        df = data[data.day==day]
        
        if (df.empty == False) and (df.act.count() > 10):
    
            X = df.iloc[:,2:14]
            Y = df.act
            
            validation_size = 0.2
            seed = 42
            
            X_train, X_validation, Y_train, Y_validation = sk.model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed, shuffle =True)

            models = []
            models.append(('LDA', LinearDiscriminantAnalysis()))
            models.append(('KNN', KNeighborsClassifier()))
            models.append(('CART', DecisionTreeClassifier()))
            models.append(('NB', GaussianNB()))
            models.append(('SVM', OneVsRestClassifier(SVC(gamma='scale'))))
            models.append(("RandomForestClassifier",OneVsRestClassifier(RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = seed))))
            results = []
            names = []
            
            for name, model in models:
                kfold = sk.model_selection.StratifiedKFold(n_splits=5, random_state=seed)
                y_pred = cross_val_predict(model, X, Y, cv = kfold)
                precision,recall,fscore,support = sk.metrics.precision_recall_fscore_support(Y, y_pred,average='weighted')
                lst_dict.append((name,user,day,precision,recall,fscore))

    COLUMN_NAMES=['model','user','day','precision','recall','fscore']
    
    # export results
    df = pd.DataFrame(data = lst_dict,  columns=COLUMN_NAMES)  
    export_csv = df.to_csv ('data/accuracy.csv', index = None, header=True) 
